# AI cập nhật dữ liệu mới nhất cho bản tin 8h30

In [66]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import import_gemini
import gemini_analyze_fuction
import gemini_comment_fuction
import gemini_summary_function
import get_and_crawl_data
import plotly_and_upload
import wichart_vietnambiz_api

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(import_gemini)
importlib.reload(gemini_analyze_fuction)
importlib.reload(gemini_comment_fuction)
importlib.reload(gemini_summary_function)
importlib.reload(get_and_crawl_data)
importlib.reload(plotly_and_upload)
importlib.reload(wichart_vietnambiz_api)

from import_default import *
from import_database import *
from import_other import *
from import_gemini import *
from gemini_analyze_fuction import *
from gemini_comment_fuction import *
from gemini_summary_function import *
from get_and_crawl_data import *
from plotly_and_upload import *
from wichart_vietnambiz_api import *

### Phần dữ liệu lịch sử các ticker

#### Dữ liệu từ Mongo và AlphaVantage

- Lấy dữ liệu từ Mongo

In [67]:
date_series = get_mongo_collection(ref_db, 'date_series')
time_series = get_mongo_collection(ref_db, 'time_series')
name_map = get_mongo_collection(ref_db, "name_map")
name_map_dict = name_map.set_index('code')['full_name'].to_dict()
full_stock_classification_df = get_mongo_collection(ref_db, 'full_stock_classification')

In [68]:
projection = {"_id": 0,"date": 1,"ticker": 1,"open": 1,"high": 1,"low": 1,"close": 1,'volume': 1}
today_index_df = get_mongo_collection(stock_db, "today_index", projection=projection)
history_index_df = get_mongo_collection(stock_db, "history_index", projection=projection)
full_index_df = pd.concat([today_index_df, history_index_df], axis=0, ignore_index=True)

other_ticker_df = get_mongo_collection(stock_db, 'other_ticker', projection=projection)
nntd_index_df = get_mongo_collection(stock_db, 'nntd_index')
nntd_stock_df = get_mongo_collection(stock_db, 'nntd_stock')

- Lấy dữ liệu từ AlphaVantage

In [69]:
# Lấy dữ liệu các tỷ giá cần thiết từ Alpha Vantage
fx_pairs = [
    ('USD', 'SEK', 'USD_SEK')
]

data_frames = []
for from_curr, to_curr, col_name in fx_pairs:
    df = get_data_from_av(from_curr, to_curr, col_name)
    if df is not None:
        data_frames.append(df)

av_ticker_df = pd.concat(data_frames, axis=1)
av_ticker_df = av_ticker_df.fillna(method='ffill').dropna()

#### Chuẩn bị các dữ liệu cần thiết

- Tính toán DXY từ các cặp tiền

In [70]:
currency_config = {
    'EUR_USD': {'source': 'other_ticker_df', 'weight': -0.576},
    'USD_JPY': {'source': 'other_ticker_df', 'weight': 0.136},
    'GBP_USD': {'source': 'other_ticker_df', 'weight': -0.119},
    'USD_CAD': {'source': 'other_ticker_df', 'weight': 0.091},
    'USD_CHF': {'source': 'other_ticker_df', 'weight': 0.036},
    'USD_SEK': {'source': 'av_ticker_df', 'weight': 0.042}
}

# Tạo DataFrame cho mỗi cặp tiền tệ
currency_data = {}
for pair, config in currency_config.items():
    if config['source'] == 'other_ticker_df':
        df = other_ticker_df[other_ticker_df['ticker'] == pair][['date', 'close']].set_index('date')
        df.columns = [pair]
    else:
        df = av_ticker_df[['USD_SEK']].copy()
        df.columns = [pair]
    
    currency_data[pair] = df

# Kết hợp tất cả dữ liệu tỷ giá
dxy_calculation_df = pd.concat(currency_data.values(), axis=1, sort=True)
dxy_calculation_df = dxy_calculation_df.sort_index().bfill().ffill()

# Tính toán chỉ số DXY theo công thức chuẩn
dxy_base = 50.14348112
dxy_calculation = dxy_base
for pair, config in currency_config.items():
    dxy_calculation *= (dxy_calculation_df[pair] ** config['weight'])

dxy_calculation_df['ticker'] = 'DXY'
dxy_calculation_df['close'] = dxy_calculation
dxy_calculation_df = dxy_calculation_df[['ticker', 'close']].reset_index()

# usd_vnd_df = av_ticker_df[['USD_VND']].rename(columns={'USD_VND': 'close'})
# usd_vnd_df['ticker'] = 'USD_VND'
# usd_vnd_df = usd_vnd_df.reset_index()

- Ghép tất cả vào ticker_dict

In [71]:
FINAL_DAYS = 20
TICKER_CONFIG = {
    # Chỉ số chứng khoán Việt Nam
    'VNINDEX':   {'df': full_index_df, 'market': 'hose', 'type': 'vn'},
    'VN30':      {'df': full_index_df, 'market': 'hose', 'type': 'vn'},
    'VN30F1M':   {'df': full_index_df, 'market': 'derivatives', 'type': 'vn'},
    
    # Chỉ số chứng khoán quốc tế
    'DJI':       {'df': other_ticker_df, 'market': 'us', 'type': 'international'},
    'FTSE':      {'df': other_ticker_df, 'market': 'eu', 'type': 'international'},
    'SSEC':      {'df': other_ticker_df, 'market': 'asia', 'type': 'international'},
    
    # Khác
    'XAU_USD':   {'df': other_ticker_df, 'market': 'commodity', 'type': 'other'},
    'CLZ':   {'df': other_ticker_df, 'market': 'commodity', 'type': 'other'},
    'DXY':       {'df': dxy_calculation_df, 'market': 'fx', 'type': 'other'},
}
FINAL_COLUMNS = ['date', 'ticker', 'close', '1d_diff', '1d_change', 'cum_change', 'market', 'type']


# 2. XỬ LÝ THEO LUỒNG MỚI
final_df_list = []
for ticker, config in TICKER_CONFIG.items():
    # Lấy và sắp xếp dữ liệu nguồn
    source_df = config['df']
    temp_df = source_df[source_df['ticker'] == ticker].sort_values('date', ascending=False).copy()

    # TÍNH TOÁN TRƯỚC trên toàn bộ chuỗi dữ liệu đã sắp xếp để đảm bảo chính xác
    temp_df['1d_diff'] = temp_df['close'][::-1].diff()[::-1].fillna(0)
    temp_df['1d_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
    
    # Gán các thông tin phân loại
    temp_df['market'] = config['market']
    temp_df['type'] = config['type']

    # CẮT 1 LẦN DUY NHẤT về 20 ngày sau khi đã tính toán xong
    temp_df = temp_df.head(FINAL_DAYS)

    # Tính toán cuối cùng trên 20 dòng đã cắt
    temp_df['cum_change'] = temp_df['1d_change'][::-1].cumsum()[::-1]

    final_df_list.append(temp_df)

# 3. KẾT QUẢ CUỐI CÙNG
daily_8h30_data_df = pd.concat(final_df_list, ignore_index=True)[FINAL_COLUMNS]


#### Phần dữ liệu tiền tệ

In [72]:
wichart_item_name_dict

{'tien_te': {'dtnh': 'Dự trữ ngoại hối',
  'ctt': 'Cung tiền tệ',
  'hd': 'Tổng tiền gửi trong nền kinh tế',
  'td': 'Tổng tín dụng vốn trong nền kinh tế',
  'lslnh': 'Lãi suất liên ngân hàng',
  'lshd': 'Lãi suất huy động',
  'lsdh': 'Lãi suất điều hành',
  'dhtg': 'Tỷ giá USD/VND'},
 'vi_mo': {'gdp': 'Tăng trưởng GDP',
  'cpi': 'Chỉ số giá tiêu dùng (CPI)',
  'iip': 'Chỉ số sản xuất công nghiệp (IIP)',
  'pmi': 'Chỉ số quản lý mua hàng (PMI)',
  'hhdv': 'Tổng mức bán lẻ hàng hóa dịch vụ',
  'cctm': 'Cán cân thương mại',
  'ncp': 'Nợ chính phủ',
  'tcns': 'Thu chi ngân sách'},
 'hang_hoa': {'heo_hơi': 'Giá heo hơi',
  'tom_the': 'Giá tôm thẻ',
  'duong': 'Giá đường',
  'soi_coton': 'Giá sợi cotton',
  'gao_tpxk': 'Giá gạo xuất khẩu',
  'quang_sat': 'Giá quặng sắt',
  'vang_the_gioi': 'Giá vàng thế giới',
  'vang': 'Giá vàng trong nước',
  'hrc_trung_quoc': 'Giá thép HRC Trung Quốc',
  'nhua_pvc_trung_quoc': 'Giá nhựa PVC Trung Quốc',
  'phot_pho': 'Giá phốt pho',
  'phan_urea_trung_qu

In [73]:
fetch_wichart_data(wichart_api_url_dict['vi_mo'] + 'dhtg')

,date,Tỷ giá USD trung tâm,Tỷ giá trần,Tỷ giá sàn,Tỷ giá USD NHTM bán ra,Tỷ USD tự do bán ra
0,2025-07-27,25182.0,26441.10,23922.90,26335.0,26450.0
1,2025-07-24,25164.0,26422.20,23905.80,26300.0,26460.0
2,2025-07-23,25166.0,26424.30,23907.70,26320.0,26460.0
3,2025-07-22,25177.0,26435.85,23918.15,26320.0,26460.0
4,2025-07-21,25179.0,26437.95,23920.05,26320.0,26450.0
...,...,...,...,...,...,...
522,2023-08-02,23803.0,24993.15,22612.85,23920.0,23760.0
523,2023-08-01,23773.0,24961.65,22584.35,23895.0,23720.0
524,2023-07-31,23757.0,24944.85,22569.15,23850.0,23700.0
525,2023-07-30,23758.0,24945.90,22570.10,23855.0,23720.0


### Phần dữ liệu cho tin tức

In [74]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
# fast_model_list = select_fast_models(get_gemini_models())
# standard_model_list = select_standard_models(get_gemini_models())
fast_model_list = ['gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']
standard_model_list = ['gemini-2.5-flash', 'gemini-2.0-flash', 'gemini-2.5-flash-lite', 'gemini-2.0-flash-lite', 'gemma-3-27b-it']

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

In [75]:
article_url_dict = {
    'VietStock': {
        'https://vietstock.vn/4222/bat-dong-san/du-an.rss': 5,
        'https://vietstock.vn/757/tai-chinh/ngan-hang.rss': 5,
        'https://vietstock.vn/775/the-gioi/kinh-te-nganh.rss': 5,
        'https://vietstock.vn/768/kinh-te/kinh-te-dau-tu.rss': 5,
        'https://vietstock.vn/830/chung-khoan/co-phieu.rss': 5,
        'https://vietstock.vn/761/kinh-te/vi-mo.rss': 5,
        'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss': 5,
        'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss': 5,
        'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss': 5,
    },
    'CafeF': {
        'https://cafef.vn/vi-mo-dau-tu.chn': 5,
        'https://cafef.vn/thi-truong-chung-khoan.chn': 10,
        'https://cafef.vn/tai-chinh-quoc-te.chn': 10,
        'https://cafef.vn/doanh-nghiep.chn': 10,
        'https://cafef.vn/bat-dong-san.chn': 5,
        'https://cafef.vn/tai-chinh-ngan-hang.chn': 5,
    },
    'Vietnambiz': {
        'https://vietnambiz.vn/tai-chinh.htm': 10,
        'https://vietnambiz.vn/quoc-te.htm': 5,
        'https://vietnambiz.vn/chung-khoan.htm': 10,
        'https://vietnambiz.vn/doanh-nghiep.htm': 10,
    },
    'VnEconomy': {
        'https://vneconomy.vn/kinh-te-the-gioi.rss': 10,
        'https://vneconomy.vn/tai-chinh.rss': 10,
        'https://vneconomy.vn/dia-oc.rss': 10,
        'https://vneconomy.vn/chung-khoan.rss': 5,
    }
}

In [76]:
# 1. Tạo dictionary cấu hình để quản lý cả 2 pattern xử lý
source_handlers = {
    'VnEconomy': {
        'type': 'batch',
        'process_batch': get_article_vneconomy 
    },
    'VietStock': {
        'type': 'item_by_item',
        'get_articles': lambda url, num: feedparser.parse(url).entries[:num],
        'get_details': lambda entry: get_article_vietstock(entry['id']),
        'get_published_time': lambda entry: getattr(entry, 'published', '') or ''
    },
    'CafeF': {
        'type': 'item_by_item',
        'get_articles': get_cafef_articles_list,
        'get_details': lambda entry: get_article_cafef(entry['id']),
        'get_published_time': lambda entry: get_cafef_published_time(entry['id'])
    },
    'Vietnambiz': {
        'type': 'item_by_item',
        'get_articles': get_vietnambiz_articles_list,
        'get_details': lambda entry: get_article_vietnambiz(entry['id']),
        'get_published_time': lambda entry: get_vietnambiz_published_time(entry['id'])
    }
}

# 2. Vòng lặp xử lý chính, giờ đây đã trở nên gọn gàng
raw_news_list = []
for source, rss_list in article_url_dict.items():
    for rss_url, num_articles in rss_list.items():
        
        handler = source_handlers.get(source)
        if not handler:
            print(f"Warning: No handler found for source '{source}'. Skipping.")
            continue

        # Phân luồng xử lý dựa trên 'type' đã định nghĩa trong handler
        if handler['type'] == 'batch':
            # Xử lý các nguồn trả về một danh sách hoàn chỉnh
            temp_news_list = handler['process_batch'](rss_url, num_articles)
            raw_news_list.extend(temp_news_list) # Dùng extend hiệu quả hơn là +

        elif handler['type'] == 'item_by_item':
            # Xử lý các nguồn cần lấy chi tiết từng tin
            feed_entries = handler['get_articles'](rss_url, num_articles)
            for entry in feed_entries:
                content, image_url = handler['get_details'](entry)
                published_time = handler['get_published_time'](entry)
                
                raw_news_list.append({
                    'source': source,
                    'title': entry['title'], 
                    'content': content,
                    'image_url': image_url,
                    'article_url': entry['id'],
                    'published_time': published_time,
                })

# 3. Chuyển đổi danh sách tin thành DataFrame
raw_news_df = pd.DataFrame(raw_news_list)
raw_news_df['published_time'] = raw_news_df['published_time'].apply(convert_published_time)

In [77]:
# Lọc và phân loại các tin nổi bật
fithered_news_df = raw_news_df.copy()
filtered_news_index_dict = get_filtered_news_index(standard_model_dict, fithered_news_df, num_articles=10)

# Tạo dictionary ánh xạ từ tên nhóm đến index của các tin nổi bật
news_type_map = {}
for k, idx_list in filtered_news_index_dict.items():
    for idx in idx_list:
        news_type_map[idx] = k

# Thêm cột news_type vào fithered_news_df và lọc ra các dòng có news_type
fithered_news_df['news_type'] = fithered_news_df.index.map(news_type_map)
fithered_news_df = fithered_news_df[fithered_news_df['news_type'].notnull()]
fithered_news_df = fithered_news_df.sort_values(by=['news_type', 'published_time'], ascending=[False, False]).reset_index(drop=True)

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_filtered_news_index.


In [78]:
# Tạo danh sách tin hàng tuần với nội dung tóm tắt
daily_8h30_news_df = fithered_news_df.copy()
daily_8h30_news_df['content'] = daily_8h30_news_df['content'].apply(lambda x: summary_daily_article(fast_model_dict, x))

# Thêm các cột cần thiết
daily_8h30_news_df['word_count'] = daily_8h30_news_df['content'].str.split().str.len()
daily_8h30_news_df['impact'] = analyze_news_impact(standard_model_dict, daily_8h30_news_df)

✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-2.5-flash-lite' thành công lần 1/2 tại hàm summary_daily_article.
✅ Model 'gemini-

In [79]:
# Lựa chọn 1 tin tức nổi bật nhất cho mỗi nhóm
daily_8h30_news_df['major_selected'] = identify_major_selected(standard_model_dict, daily_8h30_news_df)

# Lựa chọn 3 tin tức phụ cho mỗi nhóm
top_news_index_list = []
for news_type in daily_8h30_news_df['news_type'].unique():
    temp_news_index_list = get_daily_top_news(standard_model_dict, daily_8h30_news_df, news_type, num_articles = 3)
    top_news_index_list = top_news_index_list + temp_news_index_list
daily_8h30_news_df['sub_selected'] = daily_8h30_news_df.index.isin(top_news_index_list)
daily_8h30_news_df['sub_selected'] = daily_8h30_news_df['sub_selected'].apply(lambda x: 'x' if x else '')

# Gộp các tin được chọn vào cột ai_selected
daily_8h30_news_df['ai_selected'] = daily_8h30_news_df.apply(lambda x: 'o' if x['major_selected'] == 'x' else ('x' if x['sub_selected'] == 'x' else ''), axis=1)
daily_8h30_news_df = daily_8h30_news_df.drop(columns=['major_selected', 'sub_selected'])

✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm identify_major_selected.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm identify_major_selected.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm identify_major_selected.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_daily_top_news.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_daily_top_news.
✅ Model 'gemini-2.5-flash' thành công lần 1/2 tại hàm get_daily_top_news.


### Lưu dữ liệu vào MSSQL

In [80]:
daily_8h30_time_df = pd.DataFrame([{
    'time': time_series.iloc[0].item().strftime('%H:%M'),
    'date': date_series.iloc[0].item().strftime('%d/%m/%Y')
}])

In [81]:
%%capture
save_to_mssql(cts_engine, daily_8h30_data_df, 'daily_8h30_data')
save_to_mssql(cts_engine, daily_8h30_news_df, 'daily_8h30_news')
save_to_mssql(cts_engine, daily_8h30_time_df, 'daily_8h30_time')
